In [4]:
%config Completer.use_jedi = False
%config IPCompleter.greedy=True


In [5]:
# !pip3 install opencv-python

In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
from niwidgets import NiftiWidget
import nibabel as nib
import imageio
from PIL import Image
import cv2
from pathlib import Path
import glob

In [7]:
BASE_PATH=r'G:\Projects and Work\Mouse Heart Segmentation\Dataset\TAC'
subjects = os.listdir(BASE_PATH)
subjects

['TAC-#3-MicroCT-050321 - Cardiac CT_115640-diastole',
 'TAC-#3-MicroCT-050321 - Cardiac CT_115640-systole',
 'TAC-#5-MicroCT-050321 - Cardiac CT_120528-diastole',
 'TAC-#5-MicroCT-050321 - Cardiac CT_120528-systole',
 'TAC-#8-MicroCT-050321 - Cardiac CT_124052-diastole',
 'TAC-#8-MicroCT-050321 - Cardiac CT_124052-systole',
 'TAC-U2-1-MicroCT100620 - Cardiac CT_100920-diastole',
 'TAC-U2-1-MicroCT100620 - Cardiac CT_100920-systole',
 'TAC-U2-2-MicroCT100620 - Cardiac CT_101711-diastole',
 'TAC-U2-2-MicroCT100620 - Cardiac CT_101711-systole',
 'TAC-U2-3-MicroCT100620 - Cardiac CT_102620-diastole',
 'TAC-U2-3-MicroCT100620 - Cardiac CT_102620-systole']

In [8]:
def make_dir(path):
    if os.path.exists(path)==False:
        os.mkdir(path)


def normalize(data):
    data=(data-np.min(data))/(np.max(data)-np.min(data))
    return data
    
def save_axials(volume,axial='X',path=r'G:\Projects and Work\Mouse Heart Segmentation\Dataset\2D Axials'):
    X,Y,Z=volume.shape
    make_dir(os.path.join(path,subject))
    make_dir(os.path.join(path,subject,axial))
    m={'X':X,
      'Y':Y,
      'Z':Z}
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    writer = cv2.VideoWriter(os.path.join(path,subject,axial,'slides.mp4'), 
                         fourcc,
                         30, (512,512))
    for i in range(m[axial]):
        if axial=='X':
            img=volume[i,:,:].copy()
        elif axial=='Y':
            img=volume[:,i,:].copy()
        else:
            img=volume[:,:,i].copy()
        img = img*255
        img =img.astype('uint8')
        cv2.imwrite(os.path.join(path,subject,axial,str(i)+"_"+axial+".jpg"),img)
        writer.write(cv2.cvtColor(img,cv2.COLOR_GRAY2RGB))
    writer.release()

In [11]:
for subject in subjects:
    print(subject)
    files = []
    for file in os.listdir(os.path.join(BASE_PATH,subject)):
        if file.endswith(".dcm"):
            files.append(os.path.join(BASE_PATH,subject,file))
    volume = np.zeros([len(files),512,512])
    
    for i in range(len(files)):
        if str.endswith(files[i],"dcm"):
            dcm=pydicom.read_file(os.path.join(BASE_PATH,subject,files[i]))
            rescale=(dcm.RescaleIntercept)
            slope=dcm.RescaleSlope
            img=dcm.pixel_array
            volume[i,:,:]=img
            i+=1
    volume =volume *slope + (rescale)
    save_axials(normalize(volume),axial='X')
    save_axials(normalize(volume),axial='Y')
    save_axials(normalize(volume),axial='Z')
    

TAC-#3-MicroCT-050321 - Cardiac CT_115640-diastole
TAC-#3-MicroCT-050321 - Cardiac CT_115640-systole
TAC-#5-MicroCT-050321 - Cardiac CT_120528-diastole
TAC-#5-MicroCT-050321 - Cardiac CT_120528-systole
TAC-#8-MicroCT-050321 - Cardiac CT_124052-diastole
TAC-#8-MicroCT-050321 - Cardiac CT_124052-systole
TAC-U2-1-MicroCT100620 - Cardiac CT_100920-diastole
TAC-U2-1-MicroCT100620 - Cardiac CT_100920-systole
TAC-U2-2-MicroCT100620 - Cardiac CT_101711-diastole
TAC-U2-2-MicroCT100620 - Cardiac CT_101711-systole
TAC-U2-3-MicroCT100620 - Cardiac CT_102620-diastole
TAC-U2-3-MicroCT100620 - Cardiac CT_102620-systole


In [9]:
nib_seg = nib.load('abc.nii.gz').get_fdata()

In [10]:
plt.imshow(volume[:,13,:],cmap="gray")

NameError: name 'volume' is not defined

In [11]:
# from skimage import segmentation
def draw_mask(image,mask=None):
   
    rgb=np.ones([image.shape[0],image.shape[1],3])
    rgb[:,:,0]=image
    rgb[:,:,1]=image
    rgb[:,:,2]=image

    liver=mask.copy()
    liver[np.where(liver==2)]=0
    tumor=mask.copy()
    tumor[np.where(tumor==1)]=0
    tumor[np.where(tumor==2)]=1
    
    rgb = segmentation.mark_boundaries(rgb, liver, mode='thick',color=(0,0,1))
    rgb = segmentation.mark_boundaries(rgb, tumor, mode='thick',color=(1,0,0))
    rgb=rgb*255
    rgb=rgb.astype(np.uint8)
    return rgb

def show_slice_window(slice, level, window):
   max = level + window/2
   min = level - window/2
   slice = slice.clip(min,max)
   plt.figure()
   plt.imshow(slice.T, cmap="gray", origin="lower")
   plt.savefig('L'+str(level)+'W'+str(window))
    
def make_plots(images,masks=None):
    """PLot images in one row."""
    n = len(images)
    
    plt.figure(figsize=(40, 100))

    for i, (image) in enumerate(images):
        plt.subplot(1, n*2, i + 1)
        plt.xticks([])
        plt.yticks([])
        if masks is not None:
            rgb=draw_mask(image,masks[i])
        else:
            rgb=image
        plt.imshow(rgb,cmap="gray")

    plt.show()
def visualize(volume,mask=None,num_images=8,axis='Z'):
    X,Y,Z=volume.shape
    if axis=='Z':
        axials=[volume[int(i),:,:] for i in (np.linspace(0,Z,num_images,endpoint=False))]
        if mask is not None:
            segments=[mask[:,:,int(i)] for i in (np.linspace(0,Z,num_images,endpoint=False))]
        else:
            segments=None
        make_plots(axials,segments)

def make_gif(images,masks=None):
    img_arr=[]
    for i in range(len(images)):
        if masks is not None:
            rgb=draw_mask(images[i],masks[i])
        else:
            rgb=images[i].copy()
        img_arr.append(rgb)
    imageio.mimsave('movie.gif', img_arr)


In [12]:
# show_slice_window(nib_seg[0,:,:],0,150)

In [17]:
visualize((nib_seg),None,4,axis='X')

In [30]:
X,Y,Z=volume.shape
axials=[volume[int(i):,:] for i in range(0,X)]
make_gif(axials,None)

MemoryError: Unable to allocate 988. MiB for an array with shape (494, 512, 512) and data type float64